In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
# QUESTO INSTALL xlrd all'interno del virtualenv
# FONDAMENTALE!!!!
#

import sys
!{sys.executable} -m pip install xlrd

mkl-random 1.0.1 requires cython, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

WORKDIR = "csv/"
EXT = '.csv'

### Test lettura quotazioni ETF da ./csv/

In [122]:
tickers = ['A500.MI']

In [123]:
ticker_file = WORKDIR + tickers[0] + EXT

#df = pd.read_csv(ticker_file, index_col=1, parse_dates=True)
df = pd.read_csv(ticker_file, usecols=[1,2,3,4,5,6], index_col=0, parse_dates=True)

In [125]:
df.head()

,open,high,low,close,volume
dateTime,,,,,
2010-09-09,13.645,13.645,13.645,13.645,4750
2010-09-13,13.710,13.725,13.710,13.725,4790
2010-09-14,13.670,13.680,13.595,13.595,60850
2010-09-20,13.630,13.645,13.630,13.645,5850
2010-09-21,13.645,13.650,13.640,13.650,69846


In [152]:
end = df.index[df.shape[0]-1]
start = df.index[0]

end - start
end.date()

datetime.date(2019, 4, 4)

In [153]:
np.busday_count( start.date(), end.date() )

2235

### Importo la lista degli ETF borsaitalia

In [90]:
df2 = pd.read_excel('infoproviders.xlsx').iloc[6:,:]
df2.columns = df2.iloc[0,:].tolist()
df2 = df2.iloc[1:,:]
df2.set_index('N.',inplace=True)
# df2['Strumento'].unique()
# df2.drop(df2[df2['Strumento'] == 0].index, inplace=True) # pulisce
df2 = df2[df2['Strumento'] == 'ETF']
df2 = df2[['ISIN','Nome','Reuters RIC (Italy)','Indice Benchmark','TER','Area Benchmark','Emittente']]
df2.columns = ['isin','nome','ticker','benchmark','ter','areaBenchmark','emittente']
dfEtfInfo = df2

In [91]:
dfEtfInfo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 902 entries, 283 to 782
Data columns (total 7 columns):
isin             902 non-null object
nome             902 non-null object
ticker           902 non-null object
benchmark        902 non-null object
ter              902 non-null object
areaBenchmark    902 non-null object
emittente        902 non-null object
dtypes: object(7)
memory usage: 56.4+ KB


### Lista le aree degli ETF

In [92]:
aree = pd.DataFrame(dfEtfInfo.areaBenchmark.unique(),columns=['areaBenchmark'])
aree.sort_values(by='areaBenchmark',inplace=True)

aree

,areaBenchmark
30,AZIONARIO EMERGENTI - AFRICA/MEDIO ORIENTE
3,AZIONARIO EMERGENTI - ASIA
10,AZIONARIO EMERGENTI - CENTRO/SUD AMERICA
25,AZIONARIO EMERGENTI - EST EUROPA
5,AZIONARIO EMERGENTI - MONDO
18,AZIONARIO EUROPA - AREA
0,AZIONARIO EUROPA - SINGOLI PAESI
21,AZIONARIO ITALIA
4,AZIONARIO MONDO
1,AZIONARIO NORD AMERICA


In [ ]:
dfEtfInfo[dfEtfInfo.areaBenchmark == 'TITOLI DI STATO - NON EURO']

In [99]:
df3 = dfEtfInfo[dfEtfInfo.areaBenchmark == 'AZIONARIO NORD AMERICA']

In [101]:
df3.sort_values(by='ticker',ascending=True)

,isin,nome,ticker,benchmark,ter,areaBenchmark,emittente
N.,,,,,,,
711,LU1681049109,AMUNDI S&P 500 DAILY HEDGED UCITS ETF,500H.MI,S&P 500 DAILY EURO HEDGED TRN,0.0028,AZIONARIO NORD AMERICA,AMUNDI
710,LU1681048804,AMUNDI S&P 500 EUR UCITS ETF,A500.MI,S&P 500 TRN USD,0.0015,AZIONARIO NORD AMERICA,AMUNDI
739,LU1681038243,AMUNDI NASDAQ-100 UCITS ETF - EUR,ANX.MI,NASDAQ 100,0.0023,AZIONARIO NORD AMERICA,AMUNDI
781,LU1829221966,LYXOR S&P 500 BANKS UCITS ETF,BNKU.MI,S&P 500 CAP 35/20 BANKS AND DIV FINAN SEL IND NTR,0.0025,AZIONARIO NORD AMERICA,LYXOR
576,LU1130155606,UBS MSCI CANADA UCITS ETF EUR HEDGED ACC,CAHEUA.MI,MSCI CANADA 100% HEDGED TO EUR TRN,0.0043,AZIONARIO NORD AMERICA,UBS ETF
855,LU0950672807,UBS ETF MSCI CANADA UCITS ETF CAD A-ACC,CANA.MI,MSCI CANADA TRN,0.0033,AZIONARIO NORD AMERICA,UBS ETF
200,LU0446734872,UBS MSCI CANADA UCITS ETF A-DIS,CANEUA.MI,MSCI CANADA,0.0033,AZIONARIO NORD AMERICA,UBS ETF
173,IE00B52SF786,ISHARES MSCI CANADA UCITS ETF USD ACC,CSCA.MI,MSCI CANADA TRN,0.0048,AZIONARIO NORD AMERICA,ISHARES
171,IE00B53SZB19,ISHARES NASDAQ 100 UCITS ETF USD ACC,CSNDX.MI,NASDAQ 100,0.0033,AZIONARIO NORD AMERICA,ISHARES


In [119]:
df3[df3.benchmark == 'S&P 500 TRN USD']

,isin,nome,ticker,benchmark,ter,areaBenchmark,emittente
N.,,,,,,,
710,LU1681048804,AMUNDI S&P 500 EUR UCITS ETF,A500.MI,S&P 500 TRN USD,0.0015,AZIONARIO NORD AMERICA,AMUNDI
190,IE00B5BMR087,ISHARES CORE S&P 500 UCITS ETF USD ACC,CSSPX.MI,S&P 500 TRN USD,0.0007,AZIONARIO NORD AMERICA,ISHARES
961,FR0011550185,BNPP EASY S&P 500 UCITS ETF,ESEE.MI,S&P 500 TRN USD,0.0015,AZIONARIO NORD AMERICA,BNP PARIBAS
962,FR0013041530,BNPP EASY S&P 500 UCITS ETF EUR H,ESEH.MI,S&P 500 TRN USD,0.0015,AZIONARIO NORD AMERICA,BNP PARIBAS
607,IE00B5KQNG97,HSBC S&P 500 UCITS ETF,HSPD.MI,S&P 500 TRN USD,0.0009,AZIONARIO NORD AMERICA,HSBC
277,IE00B7K93397,UBS (IRL) S&P 500 UCITS ETF A-DIS,SP5EUY.MI,S&P 500 TRN USD,0.0012,AZIONARIO NORD AMERICA,UBS ETF
191,LU0496786574,LYXOR S&P 500 UCITS ETF - DIST,LYXSP.MI,S&P 500 TRN USD,0.0015,AZIONARIO NORD AMERICA,LYXOR
649,IE00BYML9W36,INVESCO S&P 500 UCITS ETF - B,SPXD.MI,S&P 500 TRN USD,0.0005,AZIONARIO NORD AMERICA,INVESCO
354,IE00B3YCGJ38,INVESCO S&P 500 UCITS ETF,SPXS.MI,S&P 500 TRN USD,0.0005,AZIONARIO NORD AMERICA,INVESCO


In [121]:
df4 = df3[df3.benchmark == 'S&P 500 TRN USD']

def check(ticker):
    ticker_file = WORKDIR + ticker + EXT
    #df = pd.read_csv(ticker_file, index_col=1, parse_dates=True)
    df = pd.read_csv(ticker_file, usecols=[1,2,3,4,5,6], index_col=0, parse_dates=True)
    end = df.index[df.shape[0]-1]
    start = df.index[0]
    
    return ticker_file

df4.ticker.apply(check)
    
    

N.
710      csv/A500.MI.csv
190     csv/CSSPX.MI.csv
961      csv/ESEE.MI.csv
962      csv/ESEH.MI.csv
607      csv/HSPD.MI.csv
277    csv/SP5EUY.MI.csv
191     csv/LYXSP.MI.csv
649      csv/SPXD.MI.csv
354      csv/SPXS.MI.csv
267      csv/SPY5.MI.csv
898      csv/VUSA.MI.csv
207      csv/XSPX.MI.csv
Name: ticker, dtype: object